### Lattices implementation

In [ ]:
# Numpy for maths
import numpy as np
import numpy.linalg as LA
from numpy.linalg import eig
# Networkx for graphing
import networkx as nx
# Scipy for solving linear systems
from scipy.linalg import solve
# Matplotlib for plots
import matplotlib.pyplot as plt

def setup_matrix(L, N):
    '''
    Set up an adjacency matrix A for the undirected nearest-neighbor 
    graph with N nodes and one shortcut from node 1 to node L.
    Inputs:
    - L: shortcut end node.
    - N: number of nodes.
    Outputs:
    - A: adjacency matrix.
    - D: degree matrix.
    - S: adjacency matrix of the subgraph obtained by removing all
    edges that are not reciprocal.
    '''
    A = np.zeros((N, N))
    
    for i in range(N-1):
        A[i, i+1] = 1
        A[i+1, i] = 1
    A[0, L-1] = 1
    A[0, -1] = 1
    A[-1, 0] = 1
    
    D = 2 * np.eye(N)
    
    S = np.copy(A)
    S[0, L-1] = 0
    
    return A, D, S

def spec_rad(A):
    '''
    Calculates the spectral radius of adjacency matrix A.
    Input:
    - A: adjacency matrix.
    Output:
    - spectral radius of matrix A.
    '''
    eigs = eig(A)[0]
    return max(abs(eigs))

def Katz(alpha, A, L=None, normed=False, approx=False):
    '''
    Calculates Katz centrality of a graph with adjacency matrix A
    and parameters t1, t2, h1 and h2 contained in the approximation
    of Katz centrality on the undirected nearest-neighbor graph 
    with shortcut.
    Inputs:
    - alpha: downweighting parameter.
    - A: adjacency matrix.
    - L: long range shortcut index, only necessary when appproximation
    parameters are requested
    - normed: boolean indicating whether the output should be normed,
    default: False.
    - approx: boolean indicating whether the approximation parameters
    for the modified ring are requested, default: False.
    Outputs:
    - centrality: Katz centrality.
    - t1 (optional): root 1.
    - t2 (optional): root 2.
    - h1 (optional): constant > 0.
    - h2 (optional): constant > 0.
    '''
    N = len(A)
    Aa = np.eye(N) - alpha*A
    rhs = np.ones(N)
    centrality = solve(Aa, rhs)
    if normed == True:
        centrality /= LA.norm(centrality, 1) 
    
    if approx==True:
     
        # Roots
        t1 = (1 - np.sqrt(1 - 4*alpha**2)) / (2*alpha)
        t2 = (1 + np.sqrt(1 - 4*alpha**2)) / (2*alpha)
    
        # solve 2x2 system to find h1 and h2
        left = np.array([[1 - 2*alpha*t1 - alpha*t1**(L-1), 1 - 2*alpha*t2 - alpha*t2**(L-1)],
                     [t1**(N/2-1) * (t1-2*alpha), t2**(N/2-1) * (t2-2*alpha)]])
        right = np.array([alpha / (1 - 2*alpha), 0])
    
        h = solve(left, right)
        h1 = float(h[0])
        h2 = float(h[1])

        return centrality, t1, t2, h1, h2
    else:
        return centrality

def NBTW_spec_rad(A, D, S):
    '''
    Calculates the spectral radius of the companion linearization
    of the reversal matrix polynomial of matrix polynomial 
    M(t) = I - At + (D-I)t^2 + (A-S)t^3. Named: C.
    Note: when dealing with an undirected network, S can be 
    replaced by A.
    Input:
    - A: adjacency matrix.
    - D: degree matrix.
    - S: adjacency matrix of the subgraph obtained by removing all
    edges that are not reciprocal.
    Output:
    - spectral radius of matrix C.
    '''
    N = len(A)
    # Create matrix C
    C = np.zeros((3*N, 3*N))
    C[0: N, 0: N] = A
    C[0: N, N: 2*N] = np.eye(N) - D
    C[0: N, 2*N: 3*N] = S - A
    C[N: 2*N, 0: N] = np.eye(N)
    C[2*N: 3*N, N: 2*N] = np.eye(N)
    
    eigs = eig(C)[0]
    return max(abs(eigs))

def NBTW(alpha, A, D, S, L=None, normed=False, approx=False):
    '''
    Calculates NBTW centrality of a graph with adjacency matrix A
    and parameters t1, t2, h1 and h2 contained in the approximation
    of NBTW centrality on the undirected nearest-neighbor graph 
    with shortcut.
    Note: when dealing with an undirected network, S can be 
    replaced by A.
    Inputs:
    - alpha: downweighting parameter.
    - A: adjacency matrix.
    - D: diagonal degree matrix.
    - S: adjacency matrix of the subgraph obtained by removing all
    edges that are not reciprocal.
    - L: long range shortcut index, only necessary when appproximation
    parameters are requested
    - normed: boolean indicating whether the output should be normed,
    default: False.
    - approx: boolean indicating whether or not the approximation 
    parameters for the modified ring are requested, default: False.
    Outputs:
    - NBTWcentrality: NBTW centrality
    - t1 (optional): root 1.
    - t2 (optional): root 2.
    - h1 (optional): constant > 0.
    - h2 (optional): constant > 0.
    '''
    N = len(A)
    
    # deformed graph Laplacian:
    M = np.eye(N) - A*alpha + (D-np.eye(N))*alpha**2  + (A-S)*alpha**3 
    
    rhs = (1-alpha**2) * np.ones(N)
    NBTWcentrality = solve(M, rhs)
    if normed == True:
        NBTWcentrality /= LA.norm(NBTWcentrality, 1) 
    
    if approx==True:
        # Roots
        t1 = alpha
        t2 = 1 / alpha
    
        # solve 2x2 system to find h1 and h2
        left = np.array([[-alpha**2 + 1 + (alpha**2-1)*alpha**L, -1 + alpha**2 + (alpha**2-1)*alpha**(2-L)], 
                     [(alpha**2 - 1) * alpha**(N/2), -2*alpha**(2 - N/2) + (1 + alpha**2)*alpha**(-N/2)]])
        right = np.array([[alpha * (alpha+1)**2], [0]])
    
        h = solve(left, right)
        h1 = float(h[0])
        h2 = float(h[1])

        return NBTWcentrality, t1, t2, h1, h2
    else:
        return NBTWcentrality

def create_graph(A, nodesize, node_color, pos=None, circular=True, with_labels=True, arrows=True):
    '''
    Plots the graph associated with adjacency matrix A. 
    Inputs:
    - A: adjacency matrix
    - nodesize: list indicating the sizes of the nodes,
    typically the centrality vector.
    - node_color: list indicating the color for each node,
    typically the centrality vector.
    - pos: dictionary containing the positions of the 
    nodes. Only necessary when not dealing with a circular
    graph.
    - circular: boolean indicating whether or not a circular
    graph is required, default: True
    - with_labels: boolean indicating whether or not labels
    should be included with the nodes, default: True
    - arrows: boolean indicating whether the edges should 
    include arrows or not, default: True
    '''
    N = len(A)
    
    G = nx.from_numpy_matrix(A, create_using=nx.MultiDiGraph)
    labeldict = {}
    for i in range(N):
        labeldict[i] = i+1
        
    nodelist = [i for i in range(N)]
        
    if circular == True:
        nx.draw_circular(G, 
                         labels=labeldict, 
                         node_size=nodesize, 
                         node_color=node_color,
                         font_color='white',
                         nodelist=nodelist,
                         with_labels=with_labels,
                         arrows=arrows)
    else:
        nx.draw(G, 
                labels=labeldict, 
                node_size=nodesize, 
                node_color=node_color,
                font_color='white',
                pos=pos,
                nodelist=nodelist,
                with_labels=with_labels,
                arrows=arrows)    
                
    def multiple_ring_setup(L, m):
    '''
    Setup an adjacency matrix A for the 3 undirected nearest-neighbor 
    graphs with shortcut. All 3 rings are connected by one central node. 
    Inputs:
    - L: shortcut end node < m.
    - m: number of nodes per ring.
    Outputs:
    - A: adjacency matrix.
    - D: degree matrix.
    - S: adjacency matrix of the subgraph obtained by removing all
    edges that are not reciprocal.
    - pos: dictionary containing the positions of the nodes.
    '''
    N = m * 3 + 1
    A = np.zeros((N, N))
    A_sq = np.matmul(A, A)
    
    D = 2 * np.eye(N)
    D[-1, -1] = 3
    
    for j in range(int(np.floor(N/m))):
        for i in range(j*m, (j+1)*m-1):
            A[i, i+1] = 1
            A[i+1, i] = 1
        A[j*m, L-1+j*m] = 1
        A[j*m, (j+1)*m-1] = 1
        A[(j+1)*m-1, j*m] = 1
        
    
        A[N-1, j*m] = 1
        A[j*m, N-1] = 1
        
        D[j*m, j*m] = 3
    S = np.copy(A)
    for j in range(int(np.floor(N/m))):
        S[j*m, L-1+j*m] = 0
      
    # Setup for the positioning of the graph nodes
    pos = {}
    for i in range(int(np.floor(N/m))):
        for x in range(m):
            if i == 0:
                pos[i*m+x] = (-1 - math.acos(math.pi/5) + math.cos(math.pi/5*x - math.pi/5),
                           1 + math.asin(math.pi/5) + math.sin(math.pi/5*x - math.pi/5))
            if i == 1:
                pos[i*m+x] = (1 + math.acos(math.pi/5) + math.cos(math.pi/5*x - 4*math.pi/5),
                          1 + math.asin(math.pi/5) + math.sin(math.pi/5*x - 4*math.pi/5))
            if i == 2:
                pos[i*m+x] = (math.cos(math.pi/5*x+math.pi/2),
                                    -2+math.sin(2*math.pi/10*x+math.pi/2))

    pos[N-1] = (0,  0)

    return A, D, S, pos

### London tube network implementation

In [ ]:
# Pandas for dataframes
import pandas as pd

# Importing the edge, node and usage data as pd.DataFrames
edges = pd.read_csv('london_tube_edges.txt', sep=" ")
nodes = pd.read_csv('london_tube_nodes.txt', sep=" ")
usage = pd.read_csv('london_tube_usage.txt', sep=" ")

def initialization(nodes, edges):
    '''
    Initialization and setup for analysis on the London tube
    network. 
    Inputs:
    - nodes: pd.DataFrame containing information on the nodes.
    - edges: pd.DataFrame contianing information on the edges.
    Outputs:
    - N: dimenion of the network.
    - G: Networkx graph of the network.
    - A: adjacency matrix.
    - D: degree matrix.
    - pos: dictionary of the postions of the stations.
    - num_edges: the number of edges in the network.
    '''
    
    N = len(nodes)

    # Create graph and matrix A
    G = nx.from_pandas_edgelist(edges, 'NodeID1', 'NodeID2')
    nodelist = list(nodes['index'])
    A = nx.to_numpy_array(G, nodelist=nodelist)

    # Create postion dictionary
    coor = nodes[['xcoor', 'ycoor']]
    pos = coor.to_dict(orient='index')

    for i in pos.keys():
        pos[i] = tuple(pos[i].values())
    
    # Create matrix D
    D = np.eye(N)
    for i in range(N):
        A_sq = A @ A
        D[i, i] = A_sq[i, i]

    num_edges = len(edges)
    
    return N, G, A, D, pos, num_edges
    

def circle_line(nodes, edges, pos):
    '''
    Setup for analysis on the London tube network circle line,
    this function needs to be called after the initialization()
    function.
    Creates a graph only containing the circle line without the
    Hammersmith & City line appendix of this line.
    Inputs:
    - nodes: pd.DataFrame containing information on the nodes.
    - edges: pd.DataFrame contianing information on the edges.
    - pos: dictionary containing the positions of all stations
    in London.
    Outputs:
    - Gc: networkx graph of the circle line.
    - pos_circle: position dictionary of the circle line stations.
    - circle_names: list with the names of the stations.
    - circle_nodes: list with the node indices of the stations.
    '''
    # Create a dictionary containing only the cirle line edges
    layer_df = edges[['LayerID', 'NodeID1', 'NodeID2']]
    circle_df = layer_df[layer_df.LayerID == 3]
    
    # Create graph
    Gc = nx.from_pandas_edgelist(circle_df, 'NodeID1', 'NodeID2', create_using=nx.MultiDiGraph)
    
    # adding the missing reciprocal edges
    add_reciprocal = list(zip(circle_df['NodeID2'], circle_df['NodeID1']))
    Gc.add_edges_from(add_reciprocal)

    # Update the position dictionary to the circle line 
    delete = [] 
    for key, val in pos.items(): 
        if key not in list(circle_df['NodeID1']) and key not in list(circle_df['NodeID2']): 
            delete.append(key) 

    pos_circle = pos.copy()
    for i in delete: 
        del pos_circle[i] 
    
    # remove circle line appendix Hammersmith & city and 2 double nodes   
    circle_appendix = [49, 314, 276, 274, 275, 280, 204, 202, 203, 313]
    Gc.remove_nodes_from(circle_appendix) 
    # Add missing reciprocal edges around appendix
    Gc.add_edge(47, 315)
    Gc.add_edge(315, 29)
    Gc.add_edge(315, 47)
    Gc.add_edge(29, 315)
    
    circle_nodes = sorted(list(Gc.nodes()))
    circle_names = nodes.iloc[circle_nodes, 1]
    

    return Gc, pos_circle, circle_names, circle_nodes